In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import skimage.io as skio
import skimage
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import tifffile as tiff
import numpy as np
from utils import *
import os
import glob
from pathlib import Path
import trackpy as tp
import scipy
import pandas as pd
from matplotlib.tri import Triangulation
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from scipy.stats import anderson
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import warnings


In [ ]:
file_directory = r"C:\Users\lizau\Desktop\walker_tracker_for_article\walker_tracker\example"
in_files = glob.glob(f'{file_directory}/*.tif')

In [ ]:
fit_method = "lq"
box_side_length = 7
drift = 0
min_gradient = 800
px_to_nm = 72
frame_to_s = 0.25

max_link_displacement_px = 2
max_gap = 0
min_tray_length = 3
min_fibre_size = 60

In [ ]:
# This cell: 
#   identifies walkers on the track
#   localizes the walkers
#   calculates the walkers' steps and trajectories
#   calculates the parameters of a 2D Gaussian distribution for the walkers' steps
#   plots the 2D distribution
#   calculates the diffusion coefficient in each dimension, average dwell time and the time the walker would need to cover the length of 1 micrometre

diff_coeff_e1 = []
diff_coeff_e2 = []
time_1000_nm = []
dwell_t = []
file_basename = []
for file in in_files:
    basename = os.path.basename(file)
    file_basename += [basename]
    basename_noext, ext = os.path.splitext(basename)
    out_dir = Path(f'{file_directory}\out_parametric')
    os.makedirs(out_dir, exist_ok=True)
    take_only_walkers_on_fibre_trajectory(file, out_dir / basename_noext, min_fibre_size=min_fibre_size)
    new_out_locs = fit_single_molecules(
        out_dir,
        basename,
        fit_method=fit_method,
        box_side_length=box_side_length,
        drift=drift,
        min_gradient=min_gradient,
        px_to_nm=px_to_nm,
    )
    locs = pd.read_hdf(new_out_locs, "locs")
    locs["mass"] = locs.photons
    
    tray = tp.link(locs, max_link_displacement_px, memory=max_gap) 
    
    # Optional: if you want to rotate the data (doesn't change the fit)
   
    '''theta_deg = 180
    theta = np.radians(theta_deg)
    
    # Rotation matrix
    R = np.array([[np.cos(theta), -np.sin(theta)],
                [np.sin(theta),  np.cos(theta)]])

    # Apply rotation
    xy = tray[["x", "y"]].values
    rotated_xy = xy @ R.T   # matrix multiply

    tray[["x", "y"]] = rotated_xy'''

    # Count the length of trajectories
    tray_by_particle = tray.groupby(["particle"])
    tray["length"] = tray_by_particle["particle"].transform("count")
    dwell_time = tray["length"].mean()

    # Exclude very short trays
    tray = tray.query(f"length>={min_tray_length}")

    # Calculate steps
    steps = tray.groupby(["particle"]).apply(get_steps_from_df)
    steps["step_len"] = np.sqrt(steps.dx**2 + steps.dy**2)

    # Save trajectories and steps
    tray_out = Path(f'{out_dir}/{basename_noext}_link{max_link_displacement_px}_traylen{min_tray_length}.tray.csv')
    tray.to_csv(tray_out)
    steps_out = Path(f'{out_dir}/{basename_noext}.steps.csv')
    steps.to_csv(steps_out)

    # Distribution_check = []
    steps_matrix = steps[['dx', 'dy']].to_numpy()
    num_columns = steps_matrix.shape[1]  
    normal_str = ''
    for var_idx in range(num_columns):
        anderson_data = anderson(steps_matrix[:, var_idx])
        is_normal = anderson_data.statistic < anderson_data.critical_values[1]
        #distribution_check += [is_normal]
        if not is_normal:
            normal = f"variable {f'dx' if var_idx == 0 else f'dy'} NOT normally distributed at {anderson_data.significance_level[1]}%\n"
        else:
            normal = f"variable {f'dx' if var_idx == 0 else f'dy'} normally distributed at {anderson_data.significance_level[1]}%\n"
        normal_str += normal 

    # Calculate the parameters of the multivariate normal distribution
    mod = MultivariateNormal()
    mod.fit(steps_matrix)

    # Calculate the e1 and e2
    cov_matrix = mod.sig_
    e1, e2, rot = cov_to_axes_and_rotation(cov_matrix)

    # Save info
    info = {}
    
    diff_info_out = Path(f'{out_dir}/{basename}.diff')
    info["average_dwell_time"] = float(dwell_time*frame_to_s)
    info["diff_long_nm_nm_s"] = float(e1*(px_to_nm**2)/(2*frame_to_s))
    info["diff_short_nm_nm_s"] = float(e2*(px_to_nm**2)/(2*frame_to_s))
    info["diff_long_nm_nm_s"] = float(e1*(px_to_nm**2)/(2*frame_to_s))
    info["diff_short_nm_nm_s"] = float(e2*(px_to_nm**2)/(2*frame_to_s))
    info["time_per_1000nm_s_long_axis"] = 1000 * 1000 / info["diff_long_nm_nm_s"] / 2
    info["mean_step_nm_s"] = float(np.sqrt(2 * info["diff_long_nm_nm_s"] * frame_to_s) / frame_to_s)
    info["normality_str"] = normal_str
    info["anderson_test"] = anderson_data
    info["mu_x"] = float(round(mod.u_[0][0], 3))
    info["mu_y"] = float(round(mod.u_[1][0], 3))
    info["sigma_x"] = float(round(cov_matrix[0][0], 3))
    info["sigma_y"] = float(round(cov_matrix[1][1], 3))
    info["sigma_xy"] = float(round(cov_matrix[0][1], 3))
    write_yaml(info, diff_info_out)

    diff_coeff_e1 += [info["diff_long_nm_nm_s"]]
    diff_coeff_e2 += [info["diff_short_nm_nm_s"]]
    time_1000_nm += [info["time_per_1000nm_s_long_axis"]]
    dwell_t += [info["average_dwell_time"]]
    
    z = mod.prob(steps_matrix)
    x = steps['dx']
    y = -steps['dy']
    triang = Triangulation(x, y)
    fig = plt.figure(figsize=(6,6))
    plt.plot(x,y, '.', alpha = 0.2)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.tricontour(triang, z.ravel(), cmap = 'viridis')
    plt.colorbar(label='probability density') 
    plt.xlabel('dx')
    plt.ylabel('dy')
    plt.title('Probability density of step sizes - contour plot')

    fig.savefig(f'{out_dir}/{basename_noext}_2d_fit.png')
    '''
    x = steps['dx'].values
    y = steps['dy'].values
    fig = plt.figure(figsize=(6, 6))
    plt.plot(x, y, '.', alpha=0.2)

    # 2. Build a regular grid
    x_grid = np.linspace(x.min()-0.5, x.max()+0.5, 200)
    y_grid = np.linspace(y.min()-0.5, y.max()+0.5, 200)
    X, Y = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([X.ravel(), Y.ravel()])

    # 3. Evaluate your model probability on the grid
    Z = mod.prob(grid_points).reshape(X.shape)

    # 4. Contour plot
    plt.contour(X, Y, Z, levels=10, cmap='viridis')
    plt.colorbar(label='probability density')

    # 5. Labels and aspect ratio
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel('dx')
    plt.ylabel('dy')
    plt.title('Probability density of step sizes - contour plot')
    fig.savefig(f'{out_dir}/{basename_noext}_2d_fit.png')'''

diff_params = {'BASENAME': file_basename, 'e1': diff_coeff_e1, 'e2': diff_coeff_e2, 'time_1000_nm': time_1000_nm, 'average_dwell_time': dwell_t}
diff_params_df = pd.DataFrame(diff_params)
diff_params_df.to_csv(f'{out_dir}/diff_info.csv')

